In [1]:
from os import listdir
from summa.summarizer import summarize
import stemmsk
import rouge
import nltk
nltk.download('punkt')

TRAINING_DIR = "data/training_summs/"

[nltk_data] Downloading package punkt to /home/luk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def stem_sentence(sentence):
    new_sentence = []
    for word in sentence.split(" "):
        new_sentence.append(stemmsk.stem(word))
    return " ".join(new_sentence)

In [4]:
stem_sentence("listina základních práv európskej únie")

'list základn práv európsk úni'

In [5]:
text = """Automatic summarization is the process of reducing a text document with a \
computer program in order to create a summary that retains the most important points \
of the original document. As the problem of information overload has grown, and as \
the quantity of data has increased, so has interest in automatic summarization. \
Technologies that can make a coherent summary take into account variables such as \
length, writing style and syntax. An example of the use of summarization technology \
is search engines such as Google. Document summarization is another."""

In [6]:
print(summarize(text))

Automatic summarization is the process of reducing a text document with a computer program in order to create a summary that retains the most important points of the original document.


In [7]:
files = listdir(TRAINING_DIR)
print(len(files))

print(files[0])
with open(TRAINING_DIR + files[0], 'r') as r:
    whole = r.read().split('\n')[:-1]
    summ, text = whole
    
print()
print(summ)
print()
print(text)

98339
f81ce708ea6c5e84b02124b75b0b239ca3f141b1.html

Mimoriadne pátranie : 16 - ročná Zlatica zmizla a nik nevie kam Polícia žiada o pomoc pri pátraní po šestnásťročnej Zlatici Kovácsovej z Veľkého Krtíša . Nezvestná Zlatica

Dievčina je nezvestná od 5 . júna , kedy odišla z miesta svojho trvalého bydliska a do dnešnej doby nepodala o sebe žiadnu správu . Nezvestná je 172 cm vysoká , štíhlej postavy , má modré oči a svetlohnedé vlasy . Naposledy mala oblečené tmavé elastické nohavice , čierne tričko s červeným obrazcom a tmavú bundu . Obuté mala čierno - ružové botasky . Akékoľvek informácie , ktoré by polícii mohli pomôcť , je možné oznámiť na ktoromkoľvek policajnom oddelení , prípadne na linke 158 .


In [15]:
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=False,
                           apply_best=False,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=False)

print('f means f1 score, r means recall, p means precision\n')
for i in range(5):
    with open(TRAINING_DIR + files[i], 'r') as r:
        whole = r.read().split('\n')[:-1]
        summ, text = whole
        tr = summarize(text).replace('\n', ' ')
        
        print('TARGET: ' + summ)
        print('TEXT RANK: ' + tr)
        
        scores = evaluator.get_scores([tr], [summ])
        print(scores)
        print() 

f means f1 score, r means recall, p means precision

TARGET: Mimoriadne pátranie : 16 - ročná Zlatica zmizla a nik nevie kam Polícia žiada o pomoc pri pátraní po šestnásťročnej Zlatici Kovácsovej z Veľkého Krtíša . Nezvestná Zlatica
TEXT RANK: Dievčina je nezvestná od 5 .
{'rouge-l': [{'f': [0.08243145848461483], 'p': [0.2246676924328796], 'r': [0.05047557202311498]}]}

TARGET: Maďarsko zdanilo banky Pred časom sme Vás informovali o snahe Maďarska zdaniť banky vo výške 0 , 4 % alebo 0 , 5 % z ich celkovej bilancie na konci roka 2009 . Maďarská vláda nakoniec schválila dodatočné zdanenie bánk a to v úsilí znížiť svoj deficit verejných financií . zástava krajiny
TEXT RANK: Mala sa zaviesť osobitná sadzba dane pre banky prostredníctvom dane z príjmu , na základe ktorej by sa bankám zvýšila sadzba dane zo súčasných 19 % na 25 % , avšak parlament túto novelu zákona o daní z príjmov neposunul do druhého čítania . Daň z príjmu tak ostáva rovnaká pre fyzické a právnické osoby vrátane bánk , kt